In [ ]:
# Iran Binomial Tree Pricing on Brent Crude Options

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Load Brent crude options data
df = pd.read_csv("../data/brent_options_data.csv")
df = df.dropna(subset=["impliedVolatility"])

# Filter for out-of-the-money call options
df = df[(df["inTheMoney"] == False) & (df["strike"] > 75)]
df = df.head(50).copy()

# Constants
S = 83.0  # Brent crude price (manual)
r = 0.05  # risk-free rate
steps = 100  # binomial tree steps

# Time to maturity in years
def get_T(expiration_date):
    exp_date = datetime.strptime(expiration_date, "%Y-%m-%d")
    delta = (exp_date - datetime.today()).days
    return max(delta / 365, 1 / 365)

df["T"] = df["expirationDate"].apply(get_T)

# Binomial Tree Model
def binomial_tree(S, K, T, r, sigma, option_type="call", steps=100):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    prices = [S * (u ** j) * (d ** (steps - j)) for j in range(steps + 1)]
    if option_type == "call":
        values = [max(0, price - K) for price in prices]
    else:
        values = [max(0, K - price) for price in prices]

    for i in range(steps - 1, -1, -1):
        values = [np.exp(-r * dt) * (p * values[j + 1] + (1 - p) * values[j]) for j in range(i + 1)]

    return values[0]

# Calculate binomial prices
df["bt_price"] = df.apply(
    lambda row: binomial_tree(S, row["strike"], row["T"], r, row["impliedVolatility"], steps=steps),
    axis=1
)

# Absolute error
df["abs_error"] = np.abs(df["bt_price"] - df["lastPrice"])

# Plot model vs. market
plt.figure(figsize=(10, 6))
plt.plot(df["strike"], df["bt_price"], label="Binomial Tree Price", color="crimson")
plt.scatter(df["strike"], df["lastPrice"], label="Market Price", color="black")
plt.xlabel("Strike Price")
plt.ylabel("Option Price")
plt.title("Brent Crude: Binomial Tree vs. Market Option Pricing (Iran Proxy)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Display comparison table
print(df[["strike", "lastPrice", "bt_price", "abs_error"]].head(10))
